# H0 Sanity Check — Colab GPU (one-click)

Этот ноутбук **автоматически** делает полный пайплайн:

- чистый `git clone` репозитория (для воспроизводимости)
- установка зависимостей
- запуск `scripts/measure_sink_text.py`
- построение графиков (`scripts/plot_*.py`)
- сохранение артефактов + `manifest.json` + `pip_freeze.txt`

## Важно про пути
Colab runtime **не имеет доступа** к локальному `/Users/...`.

Чтобы результаты "появлялись у тебя в директории без ручного копирования", включи `USE_DRIVE = True` (cell 1):
- вывод будет писаться в `Google Drive/MyDrive/sinks_runs/<RUN_NAME>/...`
- этот каталог можно синкать на Mac через Google Drive app и шарить коллаборатору.


In [29]:
# CONFIG + (optional) Google Drive + fresh clone
#
# Важно: Colab runtime НЕ видит локальный путь вида /Users/... .
# Чтобы результаты "появились у тебя локально без ручного копирования",
# сохраняй их в Google Drive (он синкается на твоём Mac через Google Drive app).

USE_DRIVE = True  # False -> сохраняем в /content и в конце качаем zip
DRIVE_SUBDIR = "sinks_runs"  # MyDrive/<DRIVE_SUBDIR>/<RUN_NAME>/...
RUN_NAME = "H0__mmlu__Qwen2.5-0.5B-Instruct__K4__qlast__N100__seed42"

REPO_URL = "https://github.com/PavelPaha/sinks-bruh.git"
REPO_DIR = "/content/sinks_repo"

TASK = "mmlu"
SPLIT = "test"
MODEL = "Qwen/Qwen2.5-0.5B-Instruct"
SAMPLES = 100
SEED = 42
SINK_TOKENS = 4
QUERY_MODE = "last"  # last | range
QUERY_START = 0
CHAT = "auto"  # auto | on | off
DEVICE = "cuda"  # cuda | cpu
QUANT = "none"  # none | 4bit | 8bit

import json
import shutil
import subprocess
import sys
from pathlib import Path

if USE_DRIVE:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=False)
    OUT_BASE = Path(f"/content/drive/MyDrive/{DRIVE_SUBDIR}/{RUN_NAME}")
else:
    OUT_BASE = Path(f"/content/{DRIVE_SUBDIR}/{RUN_NAME}")

DATA_DIR = OUT_BASE / "data"
PLOTS_DIR = OUT_BASE / "plots"
META_DIR = OUT_BASE / "meta"
for d in (DATA_DIR, PLOTS_DIR, META_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("OUT_BASE:", OUT_BASE)

# Fresh clone (reproducibility: clean repo state)
repo_path = Path(REPO_DIR)
if repo_path.exists():
    shutil.rmtree(repo_path)

subprocess.check_call(["git", "clone", REPO_URL, REPO_DIR])

# Move into repo
%cd /content/sinks_repo

GIT_HEAD = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode().strip()
print("Repo:", Path.cwd())
print("Git HEAD:", GIT_HEAD)


KeyboardInterrupt: 

In [ ]:
# Проверка GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

CUDA available: True
GPU: Tesla T4
CUDA version: 12.6


In [ ]:
# Install repo deps (reproducible) + tiny extras
# We install from pyproject.toml (editable), then add missing runtime deps if any.

!pip install -q -U pip
!pip install -q -e .

# measure_sink_text.py uses tqdm explicitly
!pip install -q tqdm

print("Python:", sys.version)
!python -c "import torch, transformers, datasets; print('torch', torch.__version__); print('transformers', transformers.__version__); print('datasets', datasets.__version__)"

# Save environment snapshot for collaborators
!pip freeze > "${META_DIR}/pip_freeze.txt"
(Path(META_DIR) / "git_head.txt").write_text(GIT_HEAD + "\n", encoding="utf-8")
print("Saved:", META_DIR / "pip_freeze.txt")
print("Saved:", META_DIR / "git_head.txt")


Cloning into 'sinks'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 69 (delta 12), reused 66 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 9.65 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/sinks/sinks/sinks


In [ ]:
# Run measurement + build plots + write manifest (end-to-end)

import glob
import time

run_started = time.time()

cmd = [
    sys.executable,
    "scripts/measure_sink_text.py",
    "--task",
    TASK,
    "--split",
    SPLIT,
    "--model",
    MODEL,
    "--samples",
    str(SAMPLES),
    "--seed",
    str(SEED),
    "--sink_tokens",
    str(SINK_TOKENS),
    "--query_mode",
    QUERY_MODE,
    "--query_start",
    str(QUERY_START),
    "--chat",
    CHAT,
    "--device",
    DEVICE,
    "--quantization",
    QUANT,
    "--out_dir",
    str(DATA_DIR),
]

print("Running:")
print(" ".join(cmd))

result = subprocess.run(cmd, capture_output=True, text=True)
print("\nSTDOUT:\n", result.stdout)
if result.stderr:
    print("\nSTDERR:\n", result.stderr)
print("\nReturn code:", result.returncode)

if result.returncode != 0:
    raise RuntimeError("measure_sink_text.py failed")

# Collect inputs
inputs = sorted(glob.glob(str(DATA_DIR / "*.jsonl.gz")))
meta = sorted(glob.glob(str(DATA_DIR / "*.meta.json")))
print("\nInputs:")
print("  jsonl.gz:", len(inputs))
print("  meta.json:", len(meta))

if not inputs:
    raise RuntimeError(f"No .jsonl.gz produced in {DATA_DIR}")

# Build plots (same as hypotheses/H0_sanity/scripts/run_plots.sh, but without uv)
plot_jobs = [
    ("scripts/plot_accuracy_vs_sink_runs.py", PLOTS_DIR / "accuracy_vs_sink", ["--inputs", *inputs]),
    ("scripts/plot_h1.py", PLOTS_DIR / "h1", ["--inputs", *inputs, "--label", "auto"]),
    ("scripts/plot_h1_heatmap.py", PLOTS_DIR / "h1_heatmap", ["--inputs", *inputs, "--label", "auto"]),
    ("scripts/plot_layer_profiles.py", PLOTS_DIR / "layers", ["--inputs", *inputs]),
    ("scripts/plot_sink_entropy.py", PLOTS_DIR / "correlation", ["--inputs", *inputs]),
]

for script, out_dir, extra_args in plot_jobs:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    p_cmd = [sys.executable, script, "--out_dir", str(out_dir), *extra_args]
    print("\nPlot:", " ".join(p_cmd))
    p = subprocess.run(p_cmd, capture_output=True, text=True)
    if p.returncode != 0:
        print("\n--- plot stdout ---\n", p.stdout)
        print("\n--- plot stderr ---\n", p.stderr)
        raise RuntimeError(f"Plot failed: {script}")

# Manifest for reproducibility
manifest = {
    "run_name": RUN_NAME,
    "repo_url": REPO_URL,
    "git_head": GIT_HEAD,
    "started_ts": run_started,
    "finished_ts": time.time(),
    "cmd": cmd,
    "params": {
        "task": TASK,
        "split": SPLIT,
        "model": MODEL,
        "samples": SAMPLES,
        "seed": SEED,
        "sink_tokens": SINK_TOKENS,
        "query_mode": QUERY_MODE,
        "query_start": QUERY_START,
        "chat": CHAT,
        "device": DEVICE,
        "quantization": QUANT,
    },
    "outputs": {
        "data_dir": str(DATA_DIR),
        "plots_dir": str(PLOTS_DIR),
        "meta_dir": str(META_DIR),
    },
}

(META_DIR / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
print("\nSaved manifest:", META_DIR / "manifest.json")
print("\nDONE. Results:")
print("  DATA_DIR:", DATA_DIR)
print("  PLOTS_DIR:", PLOTS_DIR)


STDOUT:
Loading model=Qwen/Qwen2.5-0.5B-Instruct quantization=none device=cuda...
Saved run meta to hypotheses/H0_sanity/data/mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.meta.json
Prepared 100 valid examples (requested up to 100).
Done. Wrote 100 examples to hypotheses/H0_sanity/data/mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.jsonl.gz
Accuracy: 34.00%


STDERR:
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'cais/mmlu' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
`torch_dtype` is deprecated! Use `dtype` instead!
2026-01-27 20:02:46.229912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769544166.248706    5450 cuda_dnn.cc:8579] Unable to regi

In [ ]:
# Quick sanity summary (read produced run file)

import gzip

jsonl_files = sorted(DATA_DIR.glob("*.jsonl.gz"))
meta_files = sorted(DATA_DIR.glob("*.meta.json"))

print("Meta files:", len(meta_files))
print("Data files:", len(jsonl_files))

if meta_files:
    m = json.loads(meta_files[0].read_text())
    print("\nRun meta (from file):")
    print(json.dumps(m, indent=2))

if not jsonl_files:
    raise RuntimeError("No data files")

p = jsonl_files[0]
print("\nData file:", p.name)
print("Size:", p.stat().st_size, "bytes")

n = 0
n_correct = 0
sink_vals = []
with gzip.open(p, "rt", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        r = json.loads(line)
        n += 1
        if r.get("correct") is True:
            n_correct += 1
        if r.get("sink_mass") is not None:
            sink_vals.append(float(r["sink_mass"]))

print("Rows:", n)
print("Accuracy (recomputed):", (n_correct / n) if n else None)
if sink_vals:
    print("sink_mass range:", (min(sink_vals), max(sink_vals)))


Meta files: 1
Data files: 1

Run meta:
{
  "task": "mmlu",
  "dataset": "cais/mmlu",
  "subset": "all",
  "split": "test",
  "model": "Qwen/Qwen2.5-0.5B-Instruct",
  "revision": null,
  "samples": 100,
  "seed": 42,
  "sink_tokens": 4,
  "query_mode": "last",
  "query_start": 0,
  "chat": "auto",
  "quantization": "none",
  "device": "cuda",
  "cuda_visible_devices": null,
  "transformers_version": "4.57.6",
  "torch_version": "2.9.0+cu126",
  "timestamp": 1769544172.8616776
}

📊 File size: 317122 bytes
📝 Total lines in file: 100
✅ First row keys: ['chat_mode', 'correct', 'entropy', 'hallucinated', 'idx', 'label_hallucinated', 'meta', 'model', 'pred', 'quantization', 'query_mode', 'query_start', 'seq_len', 'sink_by_layer', 'sink_by_layer_head', 'sink_mass', 'sink_tokens', 'target', 'task']
   Sample: sink_mass=0.39774010241741226, correct=True


In [ ]:
# Export results
# - If USE_DRIVE=True: artifacts are already in Google Drive (no manual copy needed).
# - If USE_DRIVE=False: create a zip and download once.

import shutil

archive_path = OUT_BASE / "run_artifacts"
zip_file = shutil.make_archive(str(archive_path), "zip", str(OUT_BASE))
print("Created:", zip_file)

if USE_DRIVE:
    print("\n✅ Saved to Google Drive:")
    print("  ", OUT_BASE)
    print("\nYou can share this Drive folder with a collaborator.")
else:
    from google.colab import files

    print("\n⬇️ Downloading zip...")
    files.download(zip_file)


📦 Creating archive...
⬇️  Downloading h0_results.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started! Extract the zip and copy files to:
   /Users/aeshef/Downloads/sinks/hypotheses/H0_sanity/data/


In [ ]:
# Reproducibility notes (for collaborators)

print("How a collaborator reproduces this run:")
print("- open this notebook")
print("- keep the same CONFIG in cell 1")
print("- run all")
print("\nWhat makes it reproducible:")
print("- fresh git clone + recorded git commit")
print("- saved META_DIR/manifest.json + META_DIR/pip_freeze.txt")
print("\nWhere to look:")
print("- manifest:", META_DIR / "manifest.json")
print("- pip freeze:", META_DIR / "pip_freeze.txt")
print("- plots:", PLOTS_DIR)
print("- data:", DATA_DIR)


📦 Found h0_results.zip
  ✓ Extracted mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.jsonl.gz
  ✓ Extracted mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.meta.json

✅ Files extracted to /content/sinks/sinks/sinks/hypotheses/H0_sanity/data

📋 Next: Download files from Colab to your local machine
   Use VS Code file sync or run the download cell below


In [ ]:
# Optional: also write a short human-readable summary next to manifest

summary = f"""# Run summary

- run_name: {RUN_NAME}
- git_head: {GIT_HEAD}
- task: {TASK}
- model: {MODEL}
- split: {SPLIT}
- samples: {SAMPLES}
- seed: {SEED}
- sink_tokens(K): {SINK_TOKENS}
- query_mode(Q): {QUERY_MODE} (query_start={QUERY_START})
- chat: {CHAT}
- device: {DEVICE}
- quantization: {QUANT}

## Outputs
- data: {DATA_DIR}
- plots: {PLOTS_DIR}
- meta: {META_DIR}

"""

(META_DIR / "SUMMARY.md").write_text(summary, encoding="utf-8")
print("Wrote:", META_DIR / "SUMMARY.md")


📥 Downloading individual files...
  ⬇️  mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.jsonl.gz (317122 bytes)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ⬇️  mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.meta.json (442 bytes)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download complete!

📋 Instructions:
   1. Files should download to your default Downloads folder
   2. Copy them to: /Users/aeshef/Downloads/sinks/hypotheses/H0_sanity/data/
   3. Or use VS Code file sync if available


In [ ]:
# Deprecated: manual download helper (not needed)
# Use cell 6 (Export results) instead.
print("Deprecated cell: use Export results cell (cell 6).")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Deprecated: Colab cannot write to /Users/... (local Mac filesystem)
# Use Google Drive (cell 1) or download the zip (cell 6).
print("Deprecated cell: Colab FS can't access /Users/... .")


cp: cannot stat '/root/Downloads/mmlu__Qwen__Qwen2.5-0.5B-Instruct__K4__qlast.*': No such file or directory
